In [ ]:
# Create a CSV file from multiple CSV provided bu HDO oscilloscope tables

import pandas as pd

first_file_no = 126
last_file_no = 7260
file_name_fmt = "oscilloscope_data/CaloG/Measure--table--{0:05d}.csv"

first_file = pd.read_csv(file_name_fmt.format(first_file_no))
first_file["fileno"] = ["", first_file_no, first_file_no]

#Set here the columms that you would like to store in the new csv file
columns_save = [ "fileno", "P1:ampl(C1)", "P2:rms(C1)", "P3:Rescale(P2)", "P4:freq(C1)", "P5:freq(C1)"]

res_df = pd.DataFrame(columns=columns_save)
res_df.loc[0] = first_file[columns_save].iloc[0]
res_df.loc[1] = first_file[columns_save].iloc[1]

counter_start = 1

for iFile in range(first_file_no+1, last_file_no+1):
    try:
        file = pd.read_csv(file_name_fmt.format(iFile))
        file["fileno"] = ["", iFile, iFile]
        res_df.loc[iFile-first_file_no+counter_start] = file[columns_save].iloc[1]
    except:
        counter_start=counter_start-1


res_df.to_csv("test_calog.csv", index=False)

In [ ]:
#reconstruct sine sweep from oscilloscope data

import pandas as pd
import numpy as np
from analysis import giveLogSpaceFarray
import matplotlib.pyplot as plt

first_file = 4664 #first file of the sequence
last_file = 5059  #last file of the sequence

tol = 0.02 # 2% tolerance on frequency set
# file name after the table processing
osc_file_name = "oscilloscope-processed/oscilloscope_calog2.csv"
out_file_name = osc_file_name[:-4] + "_analyzed_" + str(first_file) + "_" + str(last_file) + osc_file_name[-4:]
 
osc_file = pd.read_csv(osc_file_name, skiprows=[1])
sweep_data = (osc_file.loc[ (osc_file["fileno"] >= first_file) & (osc_file["fileno"] <= last_file)]).to_numpy()

frequencies = np.array(giveLogSpaceFarray(20, 2000, 50))

avg_f = []
 
for f in frequencies:
    fless = [ sweep_data[:, 4] > f*(1-tol) ]
    fmore = [ sweep_data[:, 4] < f*(1+tol) ]
    condlist = np.logical_and(fless, fmore)
    choice = [sweep_data[:, 3]]
    avg_f.append(np.sum(np.select(condlist, choice, 0))/np.count_nonzero(condlist))

avg_f = np.array(avg_f)

np.savetxt(out_file_name, np.vstack((frequencies, avg_f)).T, delimiter=",", header="freq,value(g)")

plt.figure(figsize=(20, 6), dpi=60)
plt.loglog()
plt.plot(frequencies, avg_f)
plt.show()